<a href="https://colab.research.google.com/github/analeticiagarcez/Breast_cancer_segmentation_and_classification/blob/features/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown https://cloud.andrelab.icmc.usp.br/s/egcL9TYw4mJtF2w/download/dataset.zip
!unzip dataset.zip

# Tratamento das saídas

EXPLICAÇÃO

In [4]:
# import os
# import re
# import cv2
# import pandas as pd
# from google.colab.patches import cv2_imshow

# path_dataset = '/content/Dataset_BUSI_with_GT/'
# classes = os.listdir(path_dataset)

# # Itera sobre as classes
# for c in classes:
#   # Monta o path para a pasta da classe
#   path_class = os.path.join(path_dataset, c)
#   # Pega o nome dos arquivos
#   nome_imagens = os.listdir(path_class)
#   for nome_imagem in nome_imagens:
#     if 'mask_' in nome_imagem:
#       # Escrevo o nome da nova imagem
#       nome_imagem_original = re.sub('_[0-9]+', '', nome_imagem)
#       # Cria o path das máscaras
#       path_imagem_original =  os.path.join(path_class, nome_imagem_original)
#       path_imagem_atual =  os.path.join(path_class, nome_imagem)
#       # Abre as imagens
#       img_original = cv2.imread(path_imagem_original)
#       img_atual = cv2.imread(path_imagem_atual)
#       img_somada = cv2.add(img_original, img_atual)
#       # Sobrescreve o arquivo com as imagens fundidas
#       cv2.imwrite(path_imagem_original, img_somada)

# Leitura das imagens

In [9]:
from PIL import Image
from torch.utils import data
import torchvision.transforms as transforms

class Dataset(data.Dataset):
  def __init__(self, path, image_size):
    self.files_path = path
    self.imagens = []
    self.path_imagens = []
    self.masks = []
    self.path_masks = []
    self.label = []
    self.__transform(image_size)
    #Lista as classes dos dados
    classes = os.listdir(self.files_path)
    # Itera sobre as classes
    for c in classes:
      # Monta o path para a pasta da classe
      path_class = os.path.join(self.files_path, c)
      # Pega o nome dos arquivos
      nome_imagens = os.listdir(path_class)
      for nome_imagem in nome_imagens:
        if 'mask' in nome_imagem:
          continue
        # Monta o caminho para a imagem de entrada e a mácara
        path_imagem = os.path.join(path_class, nome_imagem)
        path_mask = path_imagem.replace('.', '_mask.')
        # Faz a leitura da máscara
        imagem = Image.open(path_imagem)
        mask =  Image.open(path_mask)
        # Salva as imagens na lista
        self.imagens.append(imagem)
        self.path_imagens.append(path_imagem)
        self.masks.append(mask)
        self.path_masks.append(path_mask)
        self.label.append(c)

  def __transform(self, image_size):
    # Inicializa o transormador dos dados
    self.transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

  def __len__(self):
    return len(self.imagens)

  def __getitem__(self, idx):
    # Pega as imagens no índice
    imagem = self.imagens[idx]
    mask = self.masks[idx]
    classe = self.label[idx]
    # Transforma as imagens
    imagem = self.transform(imagem)
    mask = self.transform(mask)
    # Retorna os dados das imagens
    return {'img': imagem, 'mask': mask, 'label': classe}

In [ ]:
# import numpy as np
# import pandas as pd
# import shutil

# random_state = 42
# test_size=0.2
# #
# # copy_files
# #
# # Copia uma lista de arquivos para um diretório
# #
# # Args:
# #   indexes (list): lista de índices para serem movidos
# #   class_path (str): pasta de destino das imagens
# #
# # Return:
# #   None
# #
# def copy_files(indexes: list, class_path):
#   for i in indexes:
#     # Pega o nome dos arquivos
#     mask_name = dados.path_masks[i].split('/')[-1]
#     image_name = dados.path_imagens[i].split('/')[-1]
#     # Cria o caminho de destino
#     destiny_mask = os.path.join(class_path, mask_name)
#     destiny_image = os.path.join(class_path, image_name)
#     # Move os arquivos
#     shutil.copy(dados.path_masks[i], destiny_mask)
#     shutil.copy(dados.path_imagens[i], destiny_image)

# #Cria as pastas
# try:
#   os.mkdir('train')
#   os.mkdir('test')
# except:
#   print('As pastas já existem.')
# # Cria uma Série e pega o nome das classes
# target_series = pd.Series(dados.label)
# classes = target_series.unique()
# # Itera sobre os nomes das classes
# for c in classes:
#   # Pega as amostras da classe
#   class_samples = target_series[target_series == c]
#   # Calcula a quantidade de amostrads de teste
#   n_test_samples = len(class_samples) * 0.2
#   n_test_samples = int(n_test_samples)
#   # Pega os índices de uma amostragem dos dados
#   indexes_test = class_samples.sample(n=n_test_samples, random_state=random_state).index
#   indexes_test = indexes_test.to_list()
#   # Escreve o nome do diretorio da classe e ria as pastas
#   test_class_path = os.path.join('test',  c)
#   try:
#     os.mkdir(test_class_path)
#   except:
#     print('A pasta "test" já existe.')
#   # Copia os arquivos para a pasta de teste
#   copy_files(indexes_test, test_class_path)
#   #Cria as pastas
#   try:
#     os.mkdir('train/' + c)
#   except:
#       print('A pasta "train" já existe.')
#   # Pega os índices de treino
#   indexes_train = class_samples[~class_samples.index.isin(indexes_test)].index
#   train_class_path = test_class_path.replace('test',  'train')
#   # Copia os arquivos de treino
#   copy_files(indexes_train, train_class_path)

In [12]:
# Lê as amostras de teste e de treino
test = Dataset('/content/dataset/test', (256,256))
train = Dataset('/content/dataset/train', (256,256))

In [ ]:
pd.Series(dados.image_class).value_counts()

In [ ]:
import torchvision.models

model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=1, out_channels=1, init_features=32, pretrained=True)